# fastai training with resnet34
fastai is a great tool to create a strong baseline quickly. My idea is to convert the numpy files to images and train a model with an imagenet-pretrained model. Let's see if this approach gets us anywhere :) 

### I will smile for every upvote :) 

In [ ]:
import pandas as pd
import numpy as np
from fastai.vision.all import *
import pickle
import os
import torch

In [ ]:
train = pd.read_csv('../input/g2net-gravitational-wave-detection/training_labels.csv')
test_df = pd.read_csv('../input/g2net-gravitational-wave-detection/sample_submission.csv')

In [ ]:
train = train.sample(frac=1).reset_index(drop=True)

In [ ]:
def id2path(id, is_test):
    a, b, c = id[0], id[1], id[2]
    if is_test: return f'../input/g2net-gravitational-wave-detection/test/{a}/{b}/{c}/{id}.npy'
    return f'../input/g2net-gravitational-wave-detection/train/{a}/{b}/{c}/{id}.npy'

In [ ]:
def id2img(id, is_test):
    fname = id2path(id, is_test)
    img = np.load(fname).reshape(3,64,64).transpose(1,2,0)
    img = (img - img.min()) / (img.max() - img.min())
    return img

In [ ]:
class NumpyDataset(torch.utils.data.Dataset):
    def __init__(self, df, is_test=False):
        self.df,self.is_test = df,is_test
        
    def __getitem__(self, i):
        image_id = self.df['id'].loc[i]
        img = id2img(image_id, self.is_test).transpose(2,0,1)
        if self.is_test:
            tgt = 0 if i < 10 else 1
            return (torch.tensor(img, dtype=torch.float), torch.tensor(tgt, dtype=torch.long))
        else:
            tgt = self.df['target'].loc[i]
            return (torch.tensor(img, dtype=torch.float), torch.tensor(tgt, dtype=torch.long))
    
    def __len__(self): return len(self.df)

In [ ]:
cut = int(0.8 * len(train))
train_df = train[:cut].reset_index(drop=True)
valid_df = train[cut:].reset_index(drop=True)
len(train_df), len(valid_df)

In [ ]:
train_ds = NumpyDataset(train_df, is_test=False)
valid_ds = NumpyDataset(train_df, is_test=False)
test_ds = NumpyDataset(test_df, is_test=True)

In [ ]:
dls = DataLoaders.from_dsets(train_ds, valid_ds)
dls.c = 1

In [ ]:
learn = cnn_learner(dls, resnet34, loss_func=BCEWithLogitsLossFlat(), metrics=RocAucBinary())

In [ ]:
learn.fit_one_cycle(1, 3e-3)

In [ ]:
learn.save('model')

In [ ]:
learn.recorder.plot_loss()

# Inference

In [ ]:
test_dl = DataLoader(test_ds, bs=256, shuffle=False, drop_last=False)
preds, _ = learn.get_preds(dl=test_dl)
test_df.target = np.array(preds)
test_df.to_csv('submission.csv', index=False)
test_df.head()